In [1]:
#!huggingface-cli login --token $LoginToken

In [2]:
# !conda install datasets
# !conda install huggingface_hub
# !conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
# !pip install scikit-image
# !pip install opencv-contrib-python
# !pip install pillow

In [3]:
import torch
import torchvision.transforms
from datasets import load_dataset

''' 
Creates a class object MyDataset
************
__init__
  This function will load a part of the ImageNet-1K dataset specified in the 
  class caller.
  Options are: train, validation, and test.
************
__getitem__
  This function will return two variables containing each a variable present 
  in the original ImageNet-1K dataset.
  Before returning these variables, it will split up transforms the image to a
  fixed resolution of 256 by 256 pixels.
  
Returns: data, label
************
__len__
Returns: the length of the loaded dataset.
'''


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):

        # Loads the dataset that needs to be transformed
        self.dataset = load_dataset("imagenet-1k", split=f"{dataset}")

#         # define your transform pipline
#         transform = transforms.Compose([
#             transforms.Resize((256, 256)),
#             Lambda(lambda x: x.convert("RGB") if x.mode != "RGB" else x),  # Convert grayscale to RGB
#             transforms.ToTensor(),
#         ])
        
       # load your dataset (how every you want, this example has the dataset stored in a json file            
#         self.dataset = load_dataset("imagenet-1k", split=f"{dataset}[:1%]")

    def __getitem__(self, idx):
        # Sample row idx from the loaded dataset
        sample = self.dataset[idx]
        
        # Split up sample example into an image and label variable
        data, label = sample['image'], sample['label']
        
        transform = transforms.Compose([
            transforms.Resize((256, 256)),  # Resize to size 256x256
            Lambda(lambda x: x.convert("RGB") if x.mode != "RGB" else x),  # Convert all images to RGB format
            transforms.ToTensor(),  # Transform image to Tensor object
        ])
        
        return transform(data), torch.tensor(label)

    def __len__(self):
        return len(self.dataset)

### Setting up Multi-Layer Network

In [5]:
# import the necessary packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

In [6]:
class deCNN(Module):
    def __init__(self, numChannels, classes):
        # call the parent constructor
        super(deCNN, self).__init__()
        # initialize first set of CONV => RELU => POOL layers
        self.conv1 = Conv2d(in_channels=numChannels, out_channels=20,
            kernel_size=(5, 5))
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        # initialize second set of CONV => RELU => POOL layers
        self.conv2 = Conv2d(in_channels=20, out_channels=50,
            kernel_size=(5, 5))
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        # initialize first (and only) set of FC => RELU layers
        self.fc1 = Linear(in_features=50 * 61 * 61, out_features=500)  # Update in_features
        self.relu3 = ReLU()
        # initialize our softmax classifier
        self.fc2 = Linear(in_features=500, out_features=classes)
        self.logSoftmax = LogSoftmax(dim=1)
        
    def forward(self, x):
        # pass the input through our first set of CONV => RELU =>
        # POOL layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        # pass the output from the previous layer through the second
        # set of CONV => RELU => POOL layers
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        # flatten the output from the previous layer and pass it
        # through our only set of FC => RELU layers
        x = flatten(x, 1)
        x = self.fc1(x)
        x = self.relu3(x)
        # pass the output to our softmax classifier to get our output
        # predictions
        x = self.fc2(x)
        output = self.logSoftmax(x)
        # return the output predictions
        return output

## Training

In [7]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

In [23]:
# import the necessary packages
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import Lambda
from torch.optim import Adam
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time
from tqdm import tqdm
import glob
import os

In [9]:
# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-m", "--model", type=str, required=True,
#     help="path to output trained model")
# ap.add_argument("-p", "--plot", type=str, required=True,
#     help="path to output loss/accuracy plot")
# args = vars(ap.parse_args())

In [10]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 256
EPOCHS = 100

# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# # Define the transformation
# transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     Lambda(lambda x: x.convert("RGB") if x.mode != "RGB" else x),  # Convert grayscale to RGB
#     transforms.ToTensor(),
# ])

# # Define a function to apply the transformation to the dataset
# def transform_dataset(dataset):
#     transformed_data = []
#     for item in dataset:
#         # Apply transformation
#         img = transform(item["image"])
#         transformed_data.append({"image": img, "label": item["label"]})
#     return transformed_data

In [12]:
# # Apply the transforms directly to the datasets
# transformed_training_data = transform_dataset(training_data)
# transformed_validation_data = transform_dataset(validation_data)
# transformed_test_data = transform_dataset(test_data)

In [13]:
# # Define train and val splits if validation set is not present
# if validation_data == False:
#     TRAIN_SPLIT = 0.75
#     VAL_SPLIT = 1 - TRAIN_SPLIT

#     # Calculate the train/validation split
#     print("[INFO] generating the train/validation split...")
#     numTrainSamples = int(len(trainData) * TRAIN_SPLIT)
#     numValSamples = int(len(trainData) * VAL_SPLIT)
#     (trainData, valData) = random_split(trainData,
#         [numTrainSamples, numValSamples],
#         generator=torch.Generator().manual_seed(42))

In [14]:
train_set = MyDataset('train')
val_set = MyDataset('validation')
test_set = MyDataset('test')

# initialize the train, validation, and test data loaders
trainDataLoader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
valDataLoader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE)
testDataLoader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE)

# calculate steps per epoch for training and validation set
trainSteps = len(train_set) // BATCH_SIZE
valSteps = len(val_set) // BATCH_SIZE

Found cached dataset imagenet-1k (C:/Users/rdede/.cache/huggingface/datasets/imagenet-1k/default/1.0.0/09dbb3153f1ac686bac1f40d24f307c383b383bc171f2df5d9e91c1ad57455b9)
Found cached dataset imagenet-1k (C:/Users/rdede/.cache/huggingface/datasets/imagenet-1k/default/1.0.0/09dbb3153f1ac686bac1f40d24f307c383b383bc171f2df5d9e91c1ad57455b9)
Found cached dataset imagenet-1k (C:/Users/rdede/.cache/huggingface/datasets/imagenet-1k/default/1.0.0/09dbb3153f1ac686bac1f40d24f307c383b383bc171f2df5d9e91c1ad57455b9)


In [40]:
# initialize the deCNN model
print("[INFO] initializing the deCNN model...")
model = deCNN(
    numChannels=3,
    classes=1000).to(device)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.NLLLoss()
# initialize a dictionary to store training history
H = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": []
}

[INFO] initializing the deCNN model...


In [41]:
# Initialize model checkpoint saving location
PATH = 'models/12-20-2023'

In [42]:
# Save model checkpoint
def save_ckp(state, checkpoint_dir, epoch_nr):
    f_path = checkpoint_dir + f'/epoch_{epoch_nr}.pt'
    torch.save(state, f_path)

In [43]:
# Load model from checkpoint
print("[INFO] Loading the network from checkpoint")
files = sorted(glob.glob(f"{PATH}/*"), key=os.path.getctime, reverse=True)
checkpoint = torch.load(files[0])
model.load_state_dict(checkpoint['state_dict'])
opt.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']

[INFO] Loading the network from checkpoint


In [ ]:
###### measure how long training is going to take
print("[INFO] training the deCNN network...")
startTime = time.time()

# Create a GradScaler for mixed-precision training
scaler = GradScaler()

# loop over our epochs
for e in range(27, EPOCHS):
    # set the model in training mode
    model.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
    # initialize the number of correct predictions in the training
    # and validation step
    trainCorrect = 0
    valCorrect = 0


    # loop over the training set
    with tqdm(trainDataLoader, unit="batch") as tepoch:
        for x, y in tepoch:
            # send the input to the device
            (x, y) = (x.to(device), y.to(device))

            # perform a forward pass and calculate the training loss
            pred = model(x)
            loss = lossFn(pred, y)

            # zero out the gradients, perform the backpropagation step,
            # and update the weights
            opt.zero_grad()
            loss.backward()
            opt.step()

            # add the loss to the total training loss so far and
            # calculate the number of correct predictions
            totalTrainLoss += loss
            trainCorrect += (pred.argmax(1) == y).type(
                torch.float).sum().item()

    # switch off autograd for evaluation
    with torch.no_grad():
        # set the model in evaluation mode
        model.eval()
        # loop over the validation set
        with tqdm(valDataLoader, unit="batch") as vepoch:
            for (x, y) in vepoch:
                # send the input to the device
                (x, y) = (x.to(device), y.to(device))
                # make the predictions and calculate the validation loss
                pred = model(x)
                totalValLoss += lossFn(pred, y)
                # calculate the number of correct predictions
                valCorrect += (pred.argmax(1) == y).type(
                    torch.float).sum().item()

    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps
    # calculate the training and validation accuracy
    trainCorrect = trainCorrect / len(train_set)
    valCorrect = valCorrect / len(val_set)
    # update our training history
    H["train_loss"].append(avgTrainLoss)
    H["train_acc"].append(trainCorrect)
    H["val_loss"].append(avgValLoss)
    H["val_acc"].append(valCorrect)
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
        avgTrainLoss, trainCorrect))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
        avgValLoss, valCorrect))

    # Save the epoch model parameters
    checkpoint = {
        'epoch': e + 1,
        'state_dict': model.state_dict(),
        'optimizer': opt.state_dict()
    }

    save_ckp(checkpoint, PATH , e)

# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
    endTime - startTime))

[INFO] training the deCNN network...


 57%|█████▋    | 2836/5005 [7:07:34<5:53:08,  9.77s/batch] 

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# we can now evaluate the network on the test set
print("[INFO] evaluating network...")
# turn off autograd for testing evaluation
with torch.no_grad():
    # set the model in evaluation mode
    model.eval()

    # initialize a list to store our predictions
    preds = []
    # loop over the test set
    for batch in valDataLoader:
        x = batch["image"].to(device)
        # make the predictions and add them to the list
        pred = model(x)
        preds.extend(pred.argmax(axis=1).cpu().numpy())


# generate a classification report
print(classification_report(validation_data['label'], np.array(preds)))